In [1]:
import os
import argparse
import numpy as np
from config.default import _C as config
from config.default import update_config
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader
from torchvision import transforms
from solver import make_optimizer, WarmupMultiStepLR
from dataset.lip import LIPWithClass_Binary, LIPWithClass, LIPWithClass_Six
from net.pspnet import PSPNet
from hrnet.seg_hrnet import HighResolutionNet
from torchinfo import summary
from torchview import draw_graph
from torch.nn import functional as F
parser = argparse.ArgumentParser(description="Human Parsing")
parser.add_argument('--cfg',
                        default='config_yml/seg_hrnet_w48.yaml',
                        help='experiment configure file name',
                        type=str)
parser.add_argument('opts',
                        help="Modify config options using the command-line",
                        default=None,
                        nargs=argparse.REMAINDER)
parser.add_argument('--data_path', type=str, default='/home/jun/HumanSemanticDataset/myLIP')
parser.add_argument('--backend', type=str, default='densenet', help='Feature extractor')
parser.add_argument('--snapshot', type=str, default=None, help='Path to pre-trained weights')
parser.add_argument('--batch_size', type=int, default=16, help="Number of images sent to the network in one step.")
parser.add_argument('--epochs', type=int, default=40, help='Number of training epochs to run')
parser.add_argument('--crop_x', type=int, default=224, help='Horizontal random crop size')
parser.add_argument('--crop_y', type=int, default=224, help='Vertical random crop size')
parser.add_argument('--alpha', type=float, default=1.0, help='Coefficient for classification loss term')
parser.add_argument('--start_lr', type=float, default=0.0001, help='Learning rate')
parser.add_argument('--milestones', type=str, default='10,20,30', help='Milestones for LR decreasing')
parser.add_argument('--mode', type=str, default='all', help='Traing model: all, binary, six')
parser.add_argument('--gpu', type=int, default=0, help='List of GPUs for parallel training, e.g. 0,1,2,3')

args = parser.parse_args([])
update_config(config, args)

In [2]:
print(config.MODEL.NAME)

seg_hrnet


In [3]:
def get_seg_model(cfg, **kwargs):
    model = HighResolutionNet(cfg, **kwargs)
    model.init_weights(cfg.MODEL.PRETRAINED)
    return model

In [4]:
model = get_seg_model(config)


In [5]:
class PSPNet(nn.Module):
    def __init__(self, cfg, **kwargs):
        super().__init__()
        model = HighResolutionNet(cfg)
        #model.load_state_dict(torch.load(cfg.MODEL.PRETRAINED))
        model.init_weights(cfg.MODEL.PRETRAINED)
        self.model = model
        self.interpolate = nn.functional.interpolate
    def forward(self, x):
        x = self.model(x)
        x = self.interpolate(input=x, size=(512, 512), mode='bilinear', align_corners=config.MODEL.ALIGN_CORNERS)
        return x

In [6]:
model = PSPNet(config)

RuntimeError: Error(s) in loading state_dict for HighResolutionNet:
	Missing key(s) in state_dict: "last_layer.0.weight", "last_layer.0.bias", "last_layer.1.weight", "last_layer.1.bias", "last_layer.1.running_mean", "last_layer.1.running_var", "last_layer.3.weight", "last_layer.3.bias". 
	Unexpected key(s) in state_dict: "incre_modules.0.0.conv1.weight", "incre_modules.0.0.bn1.weight", "incre_modules.0.0.bn1.bias", "incre_modules.0.0.bn1.running_mean", "incre_modules.0.0.bn1.running_var", "incre_modules.0.0.bn1.num_batches_tracked", "incre_modules.0.0.conv2.weight", "incre_modules.0.0.bn2.weight", "incre_modules.0.0.bn2.bias", "incre_modules.0.0.bn2.running_mean", "incre_modules.0.0.bn2.running_var", "incre_modules.0.0.bn2.num_batches_tracked", "incre_modules.0.0.conv3.weight", "incre_modules.0.0.bn3.weight", "incre_modules.0.0.bn3.bias", "incre_modules.0.0.bn3.running_mean", "incre_modules.0.0.bn3.running_var", "incre_modules.0.0.bn3.num_batches_tracked", "incre_modules.0.0.downsample

In [18]:
summary(model=model, 
        input_size=(32, 3, 512, 512), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

torch.Size([32, 6, 512, 512])


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
PSPNet (PSPNet)                                              [32, 3, 512, 512]    [32, 6, 512, 512]    --                   True
├─HighResolutionNet (model)                                  [32, 3, 512, 512]    [32, 6, 128, 128]    --                   True
│    └─Conv2d (conv1)                                        [32, 3, 512, 512]    [32, 64, 256, 256]   1,728                True
│    └─SyncBatchNorm (bn1)                                   [32, 64, 256, 256]   [32, 64, 256, 256]   128                  True
│    └─ReLU (relu)                                           [32, 64, 256, 256]   [32, 64, 256, 256]   --                   --
│    └─Conv2d (conv2)                                        [32, 64, 256, 256]   [32, 64, 128, 128]   36,864               True
│    └─SyncBatchNorm (bn2)                                   [32, 64, 128, 128]   [32, 64, 128

In [20]:
x = torch.randn(1, 3, 512, 512).to('cuda')
model(x)

torch.Size([1, 6, 512, 512])


tensor([[[[-2.7439e-02, -2.7439e-02, -2.8130e-02,  ..., -2.2990e-02,
           -2.2533e-02, -2.2533e-02],
          [-2.7439e-02, -2.7439e-02, -2.8130e-02,  ..., -2.2990e-02,
           -2.2533e-02, -2.2533e-02],
          [-2.7083e-02, -2.7083e-02, -2.7920e-02,  ..., -2.2779e-02,
           -2.2413e-02, -2.2413e-02],
          ...,
          [-2.5058e-02, -2.5058e-02, -2.5725e-02,  ..., -1.0139e-02,
           -1.0411e-02, -1.0411e-02],
          [-2.4665e-02, -2.4665e-02, -2.5305e-02,  ..., -9.9808e-03,
           -1.0179e-02, -1.0179e-02],
          [-2.4665e-02, -2.4665e-02, -2.5305e-02,  ..., -9.9808e-03,
           -1.0179e-02, -1.0179e-02]],

         [[-6.9374e-03, -6.9374e-03, -6.2066e-03,  ...,  1.9447e-02,
            1.9487e-02,  1.9487e-02],
          [-6.9374e-03, -6.9374e-03, -6.2066e-03,  ...,  1.9447e-02,
            1.9487e-02,  1.9487e-02],
          [-6.6734e-03, -6.6734e-03, -5.7807e-03,  ...,  1.9012e-02,
            1.8878e-02,  1.8878e-02],
          ...,
     

: 

In [15]:
graph = draw_graph(model, x)
#graph.show()

In [10]:
class FullModel(nn.Module):
  """
  Distribute the loss on multi-gpu to reduce 
  the memory cost in the main gpu.
  You can check the following discussion.
  https://discuss.pytorch.org/t/dataparallel-imbalanced-memory-usage/22551/21
  """
  def __init__(self, model, loss):
    super(FullModel, self).__init__()
    self.model = model
    self.loss = loss

  def forward(self, inputs, labels, *args, **kwargs):
    outputs = self.model(inputs, *args, **kwargs)
    loss = self.loss(outputs, labels)
    return torch.unsqueeze(loss,0), outputs

class CrossEntropy(nn.Module):
    def __init__(self, ignore_label=-1, weight=None):
        super(CrossEntropy, self).__init__()
        self.ignore_label = ignore_label
        self.criterion = nn.CrossEntropyLoss(
            weight=weight,
            ignore_index=ignore_label
        )

    def _forward(self, score, target):
        ph, pw = score.size(2), score.size(3)
        h, w = target.size(1), target.size(2)
        if ph != h or pw != w:
            score = F.interpolate(input=score, size=(
                h, w), mode='bilinear', align_corners=config.MODEL.ALIGN_CORNERS)

        loss = self.criterion(score, target)

        return loss

    def forward(self, score, target):

        if config.MODEL.NUM_OUTPUTS == 1:
            score = [score]

        weights = config.LOSS.BALANCE_WEIGHTS
        assert len(weights) == len(score)

        return sum([w * self._forward(x, target) for (w, x) in zip(weights, score)])

In [ ]:
criterion = CrossEntropy(ignore_label=config.TRAIN.IGNORE_LABEL, weight=train_dataset.class_weights)
model = FullModel(model, criterion)